In [186]:
#conda activate AP1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from scipy.stats import skew, kurtosis

import pyfolio as pf
import empyrical as emp

In [187]:
df = pd.read_excel('data_nn.xlsx')
#df.to_pickle("data_nn.xlsx")

In [188]:
# Set the first column as the date index
df.set_index(df.columns[0], inplace=True)

# Convert the index to string and then to DatetimeIndex format
df.index = pd.to_datetime(df.index.astype(str))

# Filter the data for the last ten years
df_last_10_years = df.loc[df.index > "2020-01-02"]

# Apply rolling sum with a window of 252 and require at least 126 non-NaN values
df_rolling_sum = df_last_10_years.rolling(window=252, min_periods=int(252//2)).sum()

# Forward-fill NaN values, but limit this to a maximum of 5 consecutive fills
df_filled = df_last_10_years.ffill(limit=5)

# Drop any remaining NaN values that still exist after the forward-fill operation
df_cleaned = df_filled.dropna()

#return back original name to not interruppt code.
df_last_10_years = df_cleaned




In [189]:
def refactored_advanced_features(df_returns):
    """
    Refactored computation of advanced financial features to reduce DataFrame fragmentation.
    """
    skew = {}
    kurtosis = {}
    max_drawdown = {}
    volatility = {}
    vaR = {}
    momentum = {}
    avg_return = {}
    rsi = {}

        
        # 1. Skewness
    print("Skewness")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        skew[window] = df_returns.rolling(window).skew()

        # 2. Kurtosis
    print("Kurtosis")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        kurtosis[window]=df_returns.rolling(window).kurt()
    
    # 3. Maximum drawdown
    print("Maximum drawdown")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        max_drawdown[window] = df_returns.rolling(window).apply(emp.max_drawdown, raw=True)
    
    # 4. Volatility
    print("Volatility")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        volatility[window] = df_returns.rolling(window).std()*(252**0.5)
    
    # 5. Value at Risk
    print("Value at Risk")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        vaR[window] = df_returns.rolling(window).apply(emp.value_at_risk, raw=True)
    
    # 6. Momentum
    print("Momentum")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        momentum[window] = df_returns.rolling(window).sum() # ?

    print("Average Return")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        avg_return[window] = df_returns.rolling(window).mean()
    
    return skew, kurtosis, max_drawdown, volatility, vaR, momentum, avg_return

# This function reduces DataFrame fragmentation by constructing all columns and concatenating them at once.

# Läs tommys mex hur de gjorde reversal, sen implementera det. Fixa windows size till vad de hade i rapporten.
# skew[20].head() 

In [190]:
# Call the function and capture the output
skew, kurtosis, max_drawdown, volatility, vaR, momentum, avg_return = refactored_advanced_features(df_last_10_years)


Skewness
Kurtosis
Maximum drawdown
Volatility
Value at Risk
Momentum
Average Return


In [ ]:
# Reset the feature DataFrames list
features_df_list = []

# Create individual lists for each feature's DataFrame
skew_df_list = [] 
kurtosis_df_list = []
max_drawdown_df_list = []
volatility_df_list = []
vaR_df_list = []
momentum_df_list = []
avg_return_df_list = []

# Windows configuration
windows = [20, 40, 60, 100, 180, 240, 360, 480]

# Iterate through each feature dictionary and create a DataFrame
for feature_name, feature_dict in [('skew', skew), ('kurtosis', kurtosis), ('max_drawdown', max_drawdown), 
                                   ('volatility', volatility), ('vaR', vaR), ('momentum', momentum), ('avg_return', avg_return)]:
    # Only keep the windows that are present for each feature
    relevant_windows = windows if feature_name != 'kurtosis' else windows[:-1]
    feature_df = pd.concat({f'{feature_name}_{window}': feature_dict[window] for window in relevant_windows}, axis=1)
    
    # Append the individual DataFrame to the corresponding feature list
    if feature_name == 'skew':
        skew_df_list.append(feature_df)
    elif feature_name == 'kurtosis':
        kurtosis_df_list.append(feature_df)
    elif feature_name == 'max_drawdown':
        max_drawdown_df_list.append(feature_df)
    elif feature_name == 'volatility':
        volatility_df_list.append(feature_df)
    elif feature_name == 'vaR':
        vaR_df_list.append(feature_df)
    elif feature_name == 'momentum':
        momentum_df_list.append(feature_df)
    elif feature_name == 'avg_return':
        avg_return_df_list.append(feature_df)
    
    # Add the DataFrame to the main list
    features_df_list.append(feature_df)


# Concatenate all feature DataFrames into a single DataFrame
features_df = pd.concat(features_df_list, axis=1)

# Concatenate all feature DataFrames into a single DataFrame for each feature
if len(skew_df_list) > 1:
    skew_df = pd.concat(skew_df_list, axis=1)
if len(kurtosis_df_list) > 1:
    kurtosis_df = pd.concat(kurtosis_df_list, axis=1)
if len(max_drawdown_df_list) > 1:
    max_drawdown_df = pd.concat(max_drawdown_df_list, axis=1)
if len(volatility_df_list) > 1:
    volatility_df = pd.concat(volatility_df_list, axis=1)
if len(vaR_df_list) > 1:
    vaR_df = pd.concat(vaR_df_list, axis=1)
if len(momentum_df_list) > 1:
    momentum_df = pd.concat(momentum_df_list, axis=1)
if len(avg_return_df_list) > 1:
    avg_return_df = pd.concat(avg_return_df_list, axis=1)



# The individual lists for each feature now contain their respective DataFrames
# And features_df_list contains all the feature DataFrames
# Let's print the first item of each sublist to confirm
#print("Skew DataFrame:\n", skew_df_list[0].tail(), "\n")
#print("Kurtosis DataFrame:\n", kurtosis_df_list[0].tail(), "\n")
#print("Max Drawdown DataFrame:\n", max_drawdown_df_list[0].tail(), "\n")
#print("Volatility DataFrame:\n", volatility_df_list[0].tail(), "\n")
#print("VaR DataFrame:\n", vaR_df_list[0].tail(), "\n")
#print("Momentum DataFrame:\n", momentum_df_list[0].tail(), "\n")
#print("Average Return DataFrame:\n", avg_return_df_list[0].tail(), "\n")

# Print the last 5 rows of the combined DataFrame
features_df.tail()

In [59]:
def RSI(df_returns, window):
    """
    Computes the Relative Strength Index (RSI) for a given window.
    """
    df = df_returns.copy()
    df[df >= 0] = 1
    df[df < 0] = 0
    df = df.rolling(window).mean()*100
    return df

RSI skip for now

In [ ]:
# Initialize an empty dictionary to store the last RSI value for each window
rsi_values = {}

# Calculate RSI for each window and store the last value
for window in [20, 40, 60, 100, 180, 240, 360, 480]:
    rsi_df = RSI(df_last_10_years, window)  # df_returns is your DataFrame with returns data
    last_rsi_value = rsi_df.iloc[-1]  # Get the last row of the RSI DataFrame
    rsi_values[window] = last_rsi_value  # Store it in the dictionary with the window as the key

# Print the last RSI value for a 20-day window
print("Last RSI value for 20-day window:")
print(rsi_values[20])




Forming the DF

In [ ]:
# Define the assets and windows outside of the function for clarity
assets = [
    'Equities_0', 'Equities_1', 'Equities_2', 'Equities_3', 'Equities_4', 'Equities_5', 'Equities_6', 'Equities_7',
    'Equities_8', 'Equities_9', 'Equities_10', 'Equities_11', 'Equities_12', 'Equities_13', 'Equities_14', 'Equities_15',
    'Equities_16', 'FX_0', 'FX_1', 'FX_2', 'FX_3', 'FX_4', 'FX_5', 'FX_6', 'FX_7', 'FX_8', 'FX_9', 'FX_10', 'FX_11',
    'FX_12', 'FX_13', 'Bonds_0', 'Bonds_1', 'Bonds_2', 'Bonds_3', 'Bonds_4', 'Bonds_5', 'Bonds_6', 'Bonds_7', 'Bonds_8',
    'Bonds_9', 'Bonds_10', 'Bonds_11', 'Bonds_12', 'Bonds_13', 'Equity_Sector_0', 'Equity_Sector_1', 'Equity_Sector_2',
    'Equity_Sector_3', 'Equity_Sector_4', 'Equity_Sector_5', 'Equity_Sector_6', 'Equity_Sector_7', 'Equity_Sector_8',
    'Equity_Sector_9', 'Equity_Sector_10'
]
windows = [20, 40, 60, 100, 180, 240, 360, 480]

# Generate the final DataFrame
final_rows = []
for date in df_last_10_years.index:
    for asset in assets:
        row = [date, asset]
        for feature_name, feature_dict in [('skew', skew), ('kurtosis', kurtosis), ('max_drawdown', max_drawdown), 
                                           ('volatility', volatility), ('vaR', vaR), ('momentum', momentum), 
                                           ('avg_return', avg_return)]:
            for window in windows:
                # Check if the window exists for this feature, if not, use NaN
                value = feature_dict[window].loc[date, asset] if window in feature_dict else float('nan')
                row.append(value)
        final_rows.append(row)

# Define the column names for the final DataFrame
column_names = ['Date', 'Asset']
for feature_name in ['skew', 'kurtosis', 'max_drawdown', 'volatility', 'vaR', 'momentum', 'avg_return']:
    for window in windows:
        column_names.extend([f'{feature_name}_{window}'])

# Now create the DataFrame
final_df = pd.DataFrame(final_rows, columns=column_names)

# Show the first few rows of the DataFrame
print(final_df.tail())


NN - model free

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Define the neural network model
class MultivariateNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MultivariateNN, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.hidden_layer = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = F.leaky_relu(self.input_layer(x))
        x = F.softmax(self.hidden_layer(x))
        return x

# Custom Sharpe Ratio Loss
class SharpeRatioLoss(nn.Module):
    def __init__(self, risk_free_rate=0):
        super(SharpeRatioLoss, self).__init__()
        self.risk_free_rate = risk_free_rate

    def forward(self, outputs):
        expected_return = outputs.mean()
        std_dev_return = outputs.std()
        sharpe_ratio = (expected_return - self.risk_free_rate) / (std_dev_return + 1e-6)
        return -sharpe_ratio

# Assuming 'feature_df' is your dataset as a pandas DataFrame
# Calculated features must be part of 'feature_df'
calculated_features_df = pd.DataFrame(feature_df)
features_df = calculated_features_df.fillna(calculated_features_df.mean())

#test
returns = df_last_10_years.drop(columns=df_last_10_years.columns[0])

# Split the data into training and testing sets (considering all columns for y)
X_train, X_test, y_train, y_test = train_test_split(features_df, returns, test_size=0.2, random_state=42)

"""# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_df, features_df, test_size=0.2, random_state=42)
"""

# Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Converting to PyTorch tensor
X_train_tensor = torch.FloatTensor(X_train_scaled)
X_test_tensor = torch.FloatTensor(X_test_scaled)

# Initialize the neural network model
#model = MultivariateNN(X_train.shape[1], y_train.shape[1])
input_dim = X_train_tensor.shape[1] # Specify the number of input features
hidden_dim = 32  # Specify the number of neurons in the hidden layer
#output_dim = 55  # Specify the number of assets or allocation decisions
output_dim = y_train.shape[1]  # Number of columns to predict
print(input_dim)
model = MultivariateNN(input_dim, hidden_dim, output_dim)

# Loss and optimizer
criterion = SharpeRatioLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


# Training loop
epochs = 50
for epoch in tqdm(range(epochs)):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs)
    loss.backward()
    optimizer.step()

    # Optional: Print loss every N epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# Extracting only the weights (and biases, if needed) of the output layer
output_layer_weights = model.hidden_layer.weight.data.cpu().numpy()
output_layer_biases = model.hidden_layer.bias.data.cpu().numpy()

# You can now use output_layer_weights and output_layer_biases as needed
print("Output Layer Weights:", output_layer_weights)
print("Output Layer Biases:", output_layer_biases)

# Evaluate the model
model.eval()
with torch.no_grad():  # Turn off gradients for validation, saves memory and computations
    neural_network_output = model(X_test_tensor).cpu().numpy()
# Now neural_network_output contains the output of the neural network
print("Neural Network Output:", neural_network_output)
##print the dimensions of neural network
print("Neural Network Output shape:", neural_network_output.shape)
#with torch.no_grad():
    #train_outputs = model(X_train_tensor)
    #test_outputs = model(X_test_tensor)
    #train_loss = criterion(train_outputs)
    #test_loss = criterion(test_outputs)
    #print(f"Final Training Loss: {train_loss.item()}")
    #print(f"Final Test Loss: {test_loss.item()}")





NN - Model Based

In [214]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cvxpy as cp
import numpy as np

class RiskBudgetingLayer(nn.Module):
    def __init__(self, n_assets, risk_budgets):
        super(RiskBudgetingLayer, self).__init__()
        self.n_assets = n_assets
        self.risk_budgets = risk_budgets

    def forward(self, x):
        # Calculate covariance matrix
        covariance_matrix = torch.cov(x.T).detach().cpu().numpy()
        assert covariance_matrix.shape[0] == covariance_matrix.shape[1], "Covariance matrix is not square"

        # Ensure positive semi-definiteness of the covariance matrix
        covariance_matrix = (covariance_matrix + covariance_matrix.T) / 2
        eigenvalues, _ = np.linalg.eigh(covariance_matrix)
        covariance_matrix += np.eye(self.n_assets) * np.maximum(0, -eigenvalues.min() + 1e-6)

        # Convex optimization
        y = cp.Variable(self.n_assets)
        objective = cp.Minimize(cp.sqrt(cp.quad_form(y, covariance_matrix)))
        constraints = [cp.sum(cp.multiply(self.risk_budgets, cp.log(y))) >= 1, y >= 1e-5]
        problem = cp.Problem(objective, constraints)
        problem.solve(solver=cp.SCS, qcp=True, eps=1e-5, max_iters=100)
        optimized_allocation = y.value

        # Check for invalid values and handle them
        if optimized_allocation is None or np.any(np.isnan(optimized_allocation)):
            optimized_allocation = np.ones(self.n_assets) / self.n_assets  # Fallback to equal allocation

        # Convert back to PyTorch tensor with a differentiable operation
        optimized_allocation_tensor = torch.tensor(optimized_allocation, dtype=torch.float, requires_grad=True)

        # Apply softmax to ensure the output sums to 1
        return F.softmax(optimized_allocation_tensor * 1.0, dim=0)



class MultivariateNNWithRiskBudgeting(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_assets, risk_budgets):
        super(MultivariateNNWithRiskBudgeting, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.hidden_layer = nn.Linear(hidden_dim, hidden_dim)
        self.risk_budgeting_layer = RiskBudgetingLayer(n_assets, risk_budgets)
        
    def forward(self, x):
        x = F.leaky_relu(self.input_layer(x))
        x = F.softmax(self.hidden_layer(x), dim=1)
        x = self.risk_budgeting_layer(x)
        return x

# Assuming the data and necessary variables are already defined as per your notebook
# Example usage
input_dim = X_train_tensor.shape[1]
hidden_dim = 55
n_assets = y_train.shape[1]
risk_budgets = np.ones(n_assets) / n_assets  # Example risk budgets, equal for each asset

model = MultivariateNNWithRiskBudgeting(input_dim, hidden_dim, n_assets, risk_budgets)

# Continue with the existing training loop and loss function

# Loss and optimizer
criterion = SharpeRatioLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


# Training loop
epochs = 50
for epoch in tqdm(range(epochs)):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs)
    loss.backward()
    optimizer.step()

    # Optional: Print loss every N epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# Extracting only the weights (and biases, if needed) of the output layer
output_layer_weights = model.hidden_layer.weight.data.cpu().numpy()
output_layer_biases = model.hidden_layer.bias.data.cpu().numpy()

# You can now use output_layer_weights and output_layer_biases as needed
print("Output Layer Weights:", output_layer_weights)
print("Output Layer Biases:", output_layer_biases)

# Evaluate the model
model.eval()
with torch.no_grad():  # Turn off gradients for validation, saves memory and computations
    neural_network_output = model(X_test_tensor).cpu().numpy()
# Now neural_network_output contains the output of the neural network
print("Neural Network Output:", neural_network_output)
##print the dimensions of neural network
print("Neural Network Output shape:", neural_network_output.shape)
#with torch.no_grad():
    #train_outputs = model(X_train_tensor)
    #test_outputs = model(X_test_tensor)
    #train_loss = criterion(train_outputs)
    #test_loss = criterion(test_outputs)
    #print(f"Final Training Loss: {train_loss.item()}")
    #print(f"Final Test Loss: {test_loss.item()}")


Epoch 10/50, Loss: -147.55477905273438


Epoch 20/50, Loss: -147.55477905273438


Epoch 30/50, Loss: -147.55477905273438


Epoch 40/50, Loss: -147.55477905273438


100%|██████████| 50/50 [00:17<00:00,  2.90it/s]

Epoch 50/50, Loss: -147.55477905273438
Output Layer Weights: [[-0.03926107 -0.03149585 -0.01067632 ... -0.07414781  0.11678757
  -0.04233276]
 [-0.0853817   0.00608076 -0.06463261 ... -0.01583387 -0.03758642
   0.04945906]
 [-0.09923556 -0.05297274  0.01005682 ... -0.0130987   0.02328408
   0.00439699]
 ...
 [-0.01822511 -0.10091434  0.05556303 ... -0.02164624  0.11646841
  -0.04069964]
 [-0.12611622  0.03442544  0.06425689 ... -0.03670698  0.108124
  -0.01735552]
 [ 0.06869447 -0.12942438 -0.03528391 ... -0.06082166  0.1267455
  -0.10428334]]
Output Layer Biases: [ 0.09181866  0.05659966  0.11185624 -0.07248983  0.0737378   0.07338402
  0.12331356 -0.12485839  0.09854452  0.12545665  0.1263534   0.13361032
 -0.0551535  -0.03132666 -0.07454468  0.13317142  0.08288682 -0.10143816
  0.11570497  0.11284231  0.04390843  0.1344438   0.13156681 -0.03462477
  0.05897202 -0.04676094  0.11596219 -0.05476809  0.00488248 -0.04351601
  0.03063312  0.01747638  0.00758056  0.10790892  0.04276952  0.

Neural Network Output: [0.01804948 0.01812409 0.01820403 0.01817929 0.01820585 0.01808591
 0.0182464  0.01819765 0.01811458 0.01818441 0.01810397 0.0182211
 0.01823818 0.01818087 0.01811183 0.01823247 0.01759337 0.01823489
 0.01827067 0.01837297 0.01807944 0.01805247 0.01808746 0.01833909
 0.01810963 0.01817504 0.01808747 0.01807189 0.01815466 0.01838644
 0.01821834 0.0181925  0.01830699 0.01815723 0.01820097 0.01808427
 0.01813412 0.01822197 0.01815492 0.01821693 0.01833662 0.01821991
 0.01813872 0.01814932 0.01825572 0.01817208 0.01833373 0.01824975
 0.01825392 0.0182912  0.01828905 0.0182871  0.01806469 0.0181758
 0.01819847]
Neural Network Output shape: (55,)


Risk budgeting benchmark

AP1 risk budgetering

In [ ]:
import cvxpy as cp
from tqdm import tqdm
from joblib import Parallel, delayed
# Assuming df_last_10_years contains the daily returns
returns = df_last_10_years.drop(columns=df_last_10_years.columns[0])
n_assets = len(returns.columns)
# Define a function that will be parallelized
def optimize_weights(t, returns, n_assets):
    b = np.ones(n_assets) / n_assets  # For example, equal risk budgeting
    c = 1
    # Code that was originally in your for-loop goes here
    # For example:
    data_t = returns.iloc[:t]
    cov_matrix_values = data_t.cov().values
    cov_matrix_values = (cov_matrix_values + cov_matrix_values.T)/2
    y = cp.Variable(shape=n_assets)
    # Objective function: Minimize the square root of the portfolio variance
    objective = cp.Minimize(cp.sqrt(cp.quad_form(y, cp.psd_wrap(cov_matrix_values))))
    constraints = [
        cp.sum(cp.multiply(b, cp.log(y))) >= c,
        y >= 1e-5 #strict inequalities are not allowed
    ]
    # Formulate the optimization problem
    problem = cp.Problem(objective, constraints)
    # Solve the problem using a suitable solver
    problem.solve(solver=cp.SCS,qcp=True, eps = 1e-5, max_iters  = 100) 

    # Extract the results
    optimal_weights = y.value
    date = data_t.index[-1]
    # Return the results for this iteration
    return (date, optimal_weights)

# Precompute any variables that don't change inside the loop
# ...

# Set up the joblib parallelization
# Here, 'range(len(returns))' is the range over which you want to parallelize
# results = Parallel(n_jobs=-1)(delayed(optimize_weights)(t, returns, n_assets) for t in tqdm(range(54, len(returns))))
# Set up the joblib parallelization with tqdm
results = Parallel(n_jobs=-1)(delayed(optimize_weights)(t, returns, n_assets) for t in tqdm(range(54, len(returns), 5))
)
# After parallelization, recombine the results as necessary
# For example:
# Create a dictionary with dates as keys and optimal weights as values
optimal_weights_dict = {date: weights for date, weights in results}

w = pd.DataFrame.from_dict(optimal_weights_dict, orient='index', columns=returns.columns)
# Normalize the weights
w = w.div(w.sum(axis=1), axis=0)
# Calculate the portfolio returns
portfolio_returns = w.shift(1).mul(returns).dropna(how='all')
portfolio_returns.sum(axis=1).cumsum().plot()